<a href="https://colab.research.google.com/github/preetams7/TimeSeriesForecast/blob/main/AirportQueueForecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Parameters
start_date = datetime(2025, 8, 1)
end_date = datetime(2025, 8, 10)
interval_minutes = 15
timestamps = pd.date_range(start=start_date, end=end_date, freq=f'{interval_minutes}min')

# Festive day (e.g., Independence Day)
festive_days = [datetime(2025, 8, 15).date()]

# Data generation
data = []
for ts in timestamps:
    hour = ts.hour
    day_of_week = ts.weekday()
    is_weekend = day_of_week >= 5
    is_festive = ts.date() in festive_days
    is_rainy = np.random.rand() < 0.2  # 20% chance of rain

    # Passenger count logic
    base_passengers = 50
    if 6 <= hour <= 9 or 17 <= hour <= 20:
        base_passengers += 100  # peak hours
    if is_weekend:
        base_passengers += 30
    if is_festive:
        base_passengers += 50
    if is_rainy:
        base_passengers -= 20

    passenger_count = max(0, int(np.random.normal(base_passengers, 15)))

    # Flight buckets
    domestic_60 = np.random.poisson(2 if 6 <= hour <= 22 else 0.5)
    domestic_90 = np.random.poisson(1 if 6 <= hour <= 22 else 0.2)
    domestic_120 = np.random.poisson(1 if 6 <= hour <= 22 else 0.2)

    international_120 = np.random.poisson(1 if 8 <= hour <= 23 else 0.3)
    international_150 = np.random.poisson(1 if 8 <= hour <= 23 else 0.3)
    international_180 = np.random.poisson(1 if 8 <= hour <= 23 else 0.3)

    data.append([
        ts.strftime("%Y-%m-%d %H:%M"),
        passenger_count,
        domestic_60,
        domestic_90,
        domestic_120,
        international_120,
        international_150,
        international_180,
        is_rainy,
        'Weekend' if is_weekend else 'Weekday',
        is_festive,
        hour
    ])

# Create DataFrame
columns = [
    'timestamp', 'passenger_count', 'domestic_60min', 'domestic_90min', 'domestic_120min',
    'international_120min', 'international_150min', 'international_180min',
    'is_rainy', 'day_type', 'is_festive', 'hour_of_day'
]
df = pd.DataFrame(data, columns=columns)

# # Save to CSV
# df.to_csv("airport_traffic_data.csv", index=False)
# print("CSV file 'airport_traffic_data.csv' generated with", len(df), "rows.")


In [4]:
df.head(10)

,timestamp,passenger_count,domestic_60min,domestic_90min,domestic_120min,international_120min,international_150min,international_180min,is_rainy,day_type,is_festive,hour_of_day
0,2025-08-01 00:00,26,0,1,0,0,1,0,True,Weekday,False,0
1,2025-08-01 00:15,51,1,0,1,0,1,0,False,Weekday,False,0
2,2025-08-01 00:30,18,0,0,0,0,1,0,True,Weekday,False,0
3,2025-08-01 00:45,35,0,0,0,0,0,0,True,Weekday,False,0
4,2025-08-01 01:00,25,1,0,1,1,1,1,False,Weekday,False,1
5,2025-08-01 01:15,33,0,1,0,0,0,0,False,Weekday,False,1
6,2025-08-01 01:30,50,1,0,0,1,0,1,False,Weekday,False,1
7,2025-08-01 01:45,28,0,0,0,0,0,0,False,Weekday,False,1
8,2025-08-01 02:00,47,0,0,2,0,0,0,False,Weekday,False,2
9,2025-08-01 02:15,44,0,0,0,0,0,0,False,Weekday,False,2


In [5]:
df['is_weekend'] = df['day_type'].apply(lambda x: 0 if "Weekday" else 1)
df['is_festive'] = df['is_festive'].apply(lambda x: 1 if True else 0)
df['is_rainy'] = df['is_rainy'].apply(lambda x: 1 if True else 0)

In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split

# Sort and select target
df = df.sort_values("timestamp")
df_copy = df.copy()
df_copy = df_copy.drop(columns=["timestamp", 'day_type'])

# Normalize
scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(df_copy)

# Create sequences
sequence_length = 6
def create_io_sequence(data, sequence_length):
  X, y = [], []
  for i in range(len(data)-sequence_length):
    X.append(data[i:i+sequence_length, :]) # Slice numpy array for features
    y.append(data[i+sequence_length, 0]) # Get the target value from numpy array
  return np.array(X), np.array(y)

# Call the function with scaled_values instead of df_copy
X, y = create_io_sequence(scaled_values, sequence_length)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Build LSTM model
model = Sequential([
    LSTM(64, input_shape=(sequence_length, X_train.shape[2])), # Use X_train.shape[2] for the number of features
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

# Train
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Predict and inverse transform
predictions = model.predict(X_test)

# Need a scaler for the target variable (passenger_count) specifically
# Fit a new scaler on the original passenger_count column
target_scaler = MinMaxScaler()
target_scaler.fit(df_copy[['passenger_count']])


predicted_passengers = target_scaler.inverse_transform(predictions)
actual_passengers = target_scaler.inverse_transform(y_test.reshape(-1, 1))

# Display sample results
for i in range(5):
    print(f"Predicted: {predicted_passengers[i][0]:.1f}, Actual: {actual_passengers[i][0]:.1f}")

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 0.1111 - val_loss: 0.0382
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0319 - val_loss: 0.0261
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0244 - val_loss: 0.0262
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0230 - val_loss: 0.0231
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0203 - val_loss: 0.0236
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0212 - val_loss: 0.0213
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0174 - val_loss: 0.0209
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0200 - val_loss: 0.0199
Epoch 9/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0178 - val_loss: 0.0199
Epoch 10/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0202 - val_loss: 0.0196
Epoch 11/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0175 - val_loss: 0.0215
Epoch 12/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0177 - val_lo